# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [2]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [3]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.DataFrame(pd.read_csv('../Vacation_Search/WeatherPy_vacation.csv'))

# Display sample data
vacation_df.head(10)

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Lae,PG,75.87,overcast clouds,-6.7221,146.9847,Huon Gulf Hotel
1,1,Rikitea,PF,77.90,broken clouds,-23.1203,-134.9692,Chez Bianca & Benoit
2,3,Sorong,ID,80.64,overcast clouds,-0.8833,131.2500,Hotel Membramo Marina
3,7,Hilo,US,82.13,scattered clouds,19.7297,-155.0900,Dolphin Bay Hotel
4,10,Mar Del Plata,AR,75.22,moderate rain,-38.0023,-57.5575,Nuevo Ostende
5,16,Rawson,AR,78.46,broken clouds,-43.3002,-65.1023,Hotel Deportivo
6,20,Lethem,GY,89.65,scattered clouds,3.3803,-59.7968,Hotel Amazonas
7,23,Belawan,ID,75.49,broken clouds,3.7755,98.6832,Danau Toba International Cottage Belawan
8,24,Lorengau,PG,81.63,overcast clouds,-2.0226,147.2712,Seeadler Bay Hotel
9,26,Chuy,UY,73.58,scattered clouds,-33.6971,-53.4616,Alerces


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [5]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ['Hotel Name', 'Country', 'Description'],
    alpha = 0.7
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [22]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df['City']=='Cape Town']
vacation_end = vacation_df.loc[vacation_df['City']=='Cape Town']
vacation_stop1 = vacation_df.loc[vacation_df['City']=='Bredasdorp']
vacation_stop2 = vacation_df.loc[vacation_df['City']=='Hermanus']
vacation_stop3 = vacation_df.loc[vacation_df['City']=='Saldanha']

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [23]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start,vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], ignore_index=True)

# Display sample data
itinerary_df.head(5)

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,149,Cape Town,ZA,71.10,broken clouds,-33.9258,18.4232,Townhouse Hotel
1,86,Bredasdorp,ZA,70.97,scattered clouds,-34.5322,20.0403,Victoria Hotel
2,40,Hermanus,ZA,69.80,scattered clouds,-34.4187,19.2345,Aloe guest house
3,369,Saldanha,ZA,68.05,scattered clouds,-33.0117,17.9442,Hoedjesbaai
4,149,Cape Town,ZA,71.10,broken clouds,-33.9258,18.4232,Townhouse Hotel


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [24]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['City','Lat', 'Lng']].copy()

# Display sample data
waypoints_df

,City,Lat,Lng
0,Cape Town,-33.9258,18.4232
1,Bredasdorp,-34.5322,20.0403
2,Hermanus,-34.4187,19.2345
3,Saldanha,-33.0117,17.9442
4,Cape Town,-33.9258,18.4232


## Use GeoViews to create map that shows the four cities in the itinerary

In [25]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = 500,
    color = "City",
    hover_cols = ['City', 'Country', 'Hotel Name', 'Description'])

In [26]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [27]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [28]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '606c1e51915a4d24a9c3a61da8db8177',
 'waypoints': '-33.9258,18.4232|-34.5322,20.0403|-34.4187,19.2345|-33.0117,17.9442|-33.9258,18.4232'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [32]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [18.4232, -33.9258], 'original_index': 0},
     {'location': [20.0403, -34.5322], 'original_index': 1},
     {'location': [19.2345, -34.4187], 'original_index': 2},
     {'location': [17.9442, -33.0117], 'original_index': 3},
     {'location': [18.4232, -33.9258], 'original_index': 4}],
    'units': 'metric',
    'distance': 680492,
    'distance_units': 'meters',
    'time': 24641.346,
    'legs': [{'distance': 185245,
      'time': 6535.517,
      'steps': [{'from_index': 0,
        'to_index': 3,
        'distance': 158,
        'time': 9.492,
        'instruction': {'text': 'Drive southeast on Caledon Street.'}},
       {'from_index': 3,
        'to_index': 24,
        'distance': 390,
        'time': 32.097,
        'instruction': {'text': 'Turn left onto Buitenkant Street/M59. Continue on M59.'}},
       {'from_index': 24,
        'to_index': 30,
        'distance': 494,
        't

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [35]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response['features'][0]['geometry']['coordinates']

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [46]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        longitude.append(coordinate[0])
        latitude.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [47]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitude
route_df["latitude"] = latitude
# Display sample data
route_df.head()

,longitude,latitude
0,18.422919,-33.926062
1,18.423419,-33.926432
2,18.424143,-33.926975
3,18.424191,-33.927011
4,18.424246,-33.926958


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [39]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    height=500,
    width=700,
    line_width=3,
    color="red"
)

In [41]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)
   .Path.I   :Path   [longitude,latitude]